### Contents

01 Import libraries

02 Import datasets

03 Consistency checks

04 Merge dataframes

05 Export dataframe

01 Import libraries

In [7]:
import pandas as pd
import numpy as np
import os

02 Import datasets

In [9]:
path = r"C:\Users\cathe\OneDrive\Data Analysis\2 4 Instacart Basket Analysis\02 Data"

In [10]:
df_ords = pd.read_csv(os.path.join(path, 'Prepared Data', 'orders_checked.csv'), index_col = False)

In [11]:
df_ords_prior = pd.read_csv(os.path.join(path, 'Original Data', 'orders_products_prior.csv'), index_col = False)

In [12]:
# Check the output

df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
df_ords_prior.shape

(32434489, 4)

In [14]:
df_ords.shape

(3421083, 8)

In [15]:
df_ords.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,0,0,2539329,1,1,2,8,NaN
1,1,1,2398795,1,2,3,7,15.0
2,2,2,473747,1,3,3,12,21.0
3,3,3,2254736,1,4,4,7,29.0
4,4,4,431534,1,5,4,15,28.0


In [16]:
# Remove unintentional index columns

df_ords = df_ords.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

In [17]:
df_ords.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [18]:
df_ords.shape

(3421083, 6)

03 Consistency checks on orders_products_prior

In [42]:
# Check for mixed-type columns

for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].map(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)

No mixed-type columns

In [47]:
# Check for missing data

df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

No missing data

In [53]:
# Check for duplicates

df_dups = df_ords_prior[df_ords_prior.duplicated()]

No duplicates

In [61]:
# Check for unusual values

df_ords_prior.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


Order id starts at 2 instead of 1.  Otherwise, all values look reasonable.

04 Merge dataframes

In [22]:
# Merge orders and orders_product_prior

df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [23]:
df_merged_large.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


05 Export combined dataframe

In [25]:
df_merged_large.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_combined.pkl'))